# 0. 데이터 로드

In [39]:
from datasets import load_dataset
dataset = load_dataset("Cartinoe5930/raw_text_synthetic_dataset_50k", split = "train")

README.md:   0%|          | 0.00/360 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/40.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21900 [00:00<?, ? examples/s]

# 1. 역색인

In [40]:
from kiwipiepy import Kiwi
from collections import defaultdict
import math

In [41]:
class InvertedIndex:
    def __init__(self):
        self.reset()

    def reset(self):
        self.index = defaultdict(dict)
        self.kiwi = Kiwi()
        self.document_lengths = {}
        self.total_documents = 0
        self.average_document_length = 0
        self.documents = {}

    def tokenize(self, text):
        return [token.form for token in self.kiwi.tokenize(text)]

    def add_document(self, doc_id, question, answer, content='None'):
        tokens = self.tokenize(question)
        self.document_lengths[doc_id] = len(tokens)
        self.total_documents += 1
        self.documents[doc_id] = {'question': question, 'meta': {'answer': answer, 'content':content}}

        for token in set(tokens):
            if doc_id not in self.index[token]:
                self.index[token][doc_id] = 0
            self.index[token][doc_id] += tokens.count(token)

        self.average_document_length = sum(self.document_lengths.values()) / self.total_documents

    def calculate_bm25_score(self, query_tokens, doc_id):
        k1 = 1.5
        b = 0.75
        score = 0

        for token in query_tokens:
            if token not in self.index or doc_id not in self.index[token]:
                continue

            tf = self.index[token][doc_id]
            df = len(self.index[token])
            idf = math.log((self.total_documents - df + 0.5) / (df + 0.5) + 1)

            numerator = tf * (k1 + 1)
            denominator = tf + k1 * (1 - b + b * self.document_lengths[doc_id] / self.average_document_length)
            score += idf * numerator / denominator

        return score

    def search(self, query, k=5):
        query_tokens = self.tokenize(query)
        scores = defaultdict(float)

        for token in query_tokens:
            if token in self.index:
                for doc_id in self.index[token]:
                    scores[doc_id] += self.calculate_bm25_score(query_tokens, doc_id)

        top_k = sorted(scores.items(), key=lambda x: x[1], reverse = True)[:k]
        return [(doc_id, score, self.documents[doc_id]) for doc_id, score in top_k]

In [42]:
# add docs
from tqdm import tqdm

index = InvertedIndex()
for idx, data in enumerate(tqdm(dataset)):
    question = data['question']
    answer = data['response']
    index.add_document(idx, question, answer)

100%|██████████| 21900/21900 [00:47<00:00, 458.99it/s]


# 2. 유사 질문 10개 뽑기

In [43]:
question = dataset[10]['question']
gold_answer = dataset[10]['response']

print(question)
print("-" * 100)
print(gold_answer)

알고리즘 트레이딩과 AI 기반 트레이딩 전략의 장단점을 비교할 때, 두 전략 모두 "오버피팅"이라는 단점이 있는데, 이는 무엇을 의미하며 이로 인해 어떤 문제가 발생할 수 있는지 설명하시오.
----------------------------------------------------------------------------------------------------
오버피팅(Overfitting)은 알고리즘이나 모델이 훈련 데이터에 지나치게 잘 맞추어져 새로운 데이터에 대한 일반화 능력이 떨어지는 현상을 말합니다. 즉, 모델이 데이터의 잡음이나 특정 패턴에 과도하게 적응하여, 실세계의 변동성이나 새로운 상황에 대응하지 못하게 되는 경우입니다.

알고리즘 트레이딩과 AI 기반 트레이딩 전략에서 오버피팅이 발생하면 여러 가지 문제가 발생할 수 있습니다:

1. **실제 성과 저하**: 훈련 데이터에서 높은 성과를 기록하였더라도, 새로운 데이터에서는 성과가 급격히 떨어질 수 있습니다. 이는 투자자의 기대와 현실이 크게 차이가 나게 만들어 손실을 초래할 수 있습니다.

2. **위험 관리의 실패**: 오버피팅된 모델은 특정한 시장 환경에서만 잘 작동하게 되어, 변동성이 큰 시장이나 예기치 않은 사건에 대한 대응력이 떨어집니다. 따라서 이로 인해 투자 포트폴리오의 위험이 증가할 수 있습니다.

3. **신뢰성 저하**: 오버피팅된 모델은 주어진 데이터 세트를 잘 설명하더라도, 시장 상황이 바뀔 경우 신뢰할 수 있는 결과를 제공하지 못합니다. 이는 투자자들이 해당 트레이딩 전략을 중단하게 만들거나 다른 전략으로 전환하게 만들 수 있습니다.

결론적으로, 알고리즘 트레이딩과 AI 기반 트레이딩 전략 모두에서 오버피팅은 데이터 기반 의사결정의 신뢰성을 해치고 실질적인 손실을 초래하는 중요한 단점 중 하나입니다. 이러한 문제를 방지하기 위해 모델을 개발할 때는 교차 검증, 정규화 기법 등을 사용하여 과적합을 방지하는 방법이 필요합니다.


In [44]:
results = index.search(question, k=10)

In [45]:
for item in results:
    print(item[2]['question'])
    print("-" * 100)

알고리즘 트레이딩과 AI 기반 트레이딩 전략의 장단점을 비교할 때, 두 전략 모두 "오버피팅"이라는 단점이 있는데, 이는 무엇을 의미하며 이로 인해 어떤 문제가 발생할 수 있는지 설명하시오.
----------------------------------------------------------------------------------------------------
알고리즘 트레이딩과 AI 기반 트레이딩 전략의 장점과 단점을 비교할 때, 어떤 두 가지 요소가 알고리즘 트레이딩의 한계를 설명하는 데 중요한 역할을 하는지 설명하고, 이러한 요소들이 AI 기반 트레이딩 전략과 어떻게 차별화되는지를 논의하시오.
----------------------------------------------------------------------------------------------------
Renaissance Technologies와 Citadel Securities 모두 AI 기반 거래 전략을 사용하고 있는데, 이 두 펀드가 사용하는 AI 알고리즘의 특징과 그로 인해 발생할 수 있는 문제점은 무엇인지 설명해보세요.
----------------------------------------------------------------------------------------------------
알고리즘 트레이딩을 통해 거래를 자동으로 수행할 때, 매매 전략의 개발에서 가장 중요한 요소는 무엇이며, 이러한 전략이 규제 변경과 어떤 관계가 있는지를 설명하시오.
----------------------------------------------------------------------------------------------------
AI 기반 트레이딩 전략이 1980년대부터 활용되고 있는 Renaissance Technologies의 Medallion 전략은 어떤 특징을 가지고 있으며, 이 전략과 다른 설정에서 AI 알고리즘을 사용하는 Citadel Securi

# 3. 오답 생성하기

In [46]:

import openai
from dotenv import load_dotenv
import os

dotenv_path = os.path.join(os.getcwd(),'KRX', '.env')
load_dotenv(dotenv_path)

openai_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI(api_key=openai_key)

In [47]:
wrong_answer_system_prompt = "You will given a question, gold answer and one irrelevant answer. Your job is to transform the irrelvant answer to a well-designed wrong answer. Replace mentions of different companies to the company of interest in the question. After the detail a bit to make good wrong answer. please write in korean"

wrong_answer_user_prompt = """
### Question: {}
### Gold Answer: {}
### irrelevant Answer: {}
### Good Wrong Answer:
"""

In [49]:
# 8.42 -> 8.29 : 0.13  - 4o
# 8.29 -> 8.27 : 0.02  - 3.5-turbo
selections = []

for item in results:
    irrelevant_answer = item[2]['meta']['answer']
    prompt = wrong_answer_user_prompt.format(question, gold_answer, irrelevant_answer)

    instruction_msg = {"role":"system", "content": wrong_answer_system_prompt}
    user_msg = {"role":'user', "content":prompt}

    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages = [instruction_msg, user_msg]
    )

    selections.append(response.choices[0].message.content)

selections

['알고리즘 트레이딩과 AI 기반 트레이딩 전략에서 오버피팅이 발생하게 되면, 트레이딩 회사는 두 가지 주요 문제를 직면하게 됩니다:\n\n1. **고객 혼란 증가**: 오버피팅된 모델은 예기치 못한 상황에 대처하지 못하고, 실제 거래에서 과도한 리스크를 노출시킬 수 있습니다. 이는 트레이딩 서비스의 신뢰성을 훼손시키고 고객들 사이에서 혼란을 초래할 수 있습니다.\n\n2. **시장 지위 하락**: 오버피팅으로 인해 발생하는 예기치 못한 손실은 트레이딩 회사의 시장에서의 평판을 하락시킬 수 있습니다. 이는 경쟁사에 대한 약점이 되어 이익 감소로 이어질 수 있습니다.\n\n이러한 문제는 트레이딩 회사가 정보 조작, 과도한 트레이딩 등으로부터 회피하는 노력을 강화해야 함을 시사합니다.',
 "알고리즘 트레이딩과 AI 기반 트레이딩 전략을 비교할 때, 함께 고려해야 할 중요한 측면은 '사용자 친화성'과 '안정성'입니다.\n\n1. **사용자 친화성**:\n   TED Company의 AI 기반 트레이딩은 뛰어난 사용자 친화성을 제공합니다. TED의 플랫폼은 직관적인 인터페이스를 통해 사용자가 쉽게 전략을 설정하고 조정할 수 있습니다. 또한 TED는 사용자 요구에 따라 맞춤 설정을 제공하며, 비기술적인 투자자도 쉽게 활용할 수 있는 기능을 제공합니다.\n\n   한편, GOOGLE Company의 알고리즘 트레이딩은 사용자들에게 조정할 수 있는 유연한 설정이 제한적입니다. GOOGLE의 시스템은 복잡하고 기술적인 요구사항을 가지고 있어, 일반 사용자가 쉽게 다루기 어려울 수 있습니다.\n\n2. **안정성**:\n    TED Company의 AI 기반 트레이딩은 안정성이 뛰어나다고 알려져 있습니다. TED의 시스템은 다양한 상황에서도 안정적으로 작동하여 예기치 않은 문제나 중단 없이 신속하게 대응할 수 있습니다.\n\n    반면 MICROSOFT Company의 알고리즘 트레이딩은 가끔씩 불안정한 상황을 보일 수 있습니다. MICROSOFT의 시스템은 사용 중 갑

In [50]:
print(question)
print()
for idx, item in enumerate(selections):
    print(idx+1, end='')
    print("-" * 100)
    print(item)

알고리즘 트레이딩과 AI 기반 트레이딩 전략의 장단점을 비교할 때, 두 전략 모두 "오버피팅"이라는 단점이 있는데, 이는 무엇을 의미하며 이로 인해 어떤 문제가 발생할 수 있는지 설명하시오.

1----------------------------------------------------------------------------------------------------
알고리즘 트레이딩과 AI 기반 트레이딩 전략에서 오버피팅이 발생하게 되면, 트레이딩 회사는 두 가지 주요 문제를 직면하게 됩니다:

1. **고객 혼란 증가**: 오버피팅된 모델은 예기치 못한 상황에 대처하지 못하고, 실제 거래에서 과도한 리스크를 노출시킬 수 있습니다. 이는 트레이딩 서비스의 신뢰성을 훼손시키고 고객들 사이에서 혼란을 초래할 수 있습니다.

2. **시장 지위 하락**: 오버피팅으로 인해 발생하는 예기치 못한 손실은 트레이딩 회사의 시장에서의 평판을 하락시킬 수 있습니다. 이는 경쟁사에 대한 약점이 되어 이익 감소로 이어질 수 있습니다.

이러한 문제는 트레이딩 회사가 정보 조작, 과도한 트레이딩 등으로부터 회피하는 노력을 강화해야 함을 시사합니다.
2----------------------------------------------------------------------------------------------------
알고리즘 트레이딩과 AI 기반 트레이딩 전략을 비교할 때, 함께 고려해야 할 중요한 측면은 '사용자 친화성'과 '안정성'입니다.

1. **사용자 친화성**:
   TED Company의 AI 기반 트레이딩은 뛰어난 사용자 친화성을 제공합니다. TED의 플랫폼은 직관적인 인터페이스를 통해 사용자가 쉽게 전략을 설정하고 조정할 수 있습니다. 또한 TED는 사용자 요구에 따라 맞춤 설정을 제공하며, 비기술적인 투자자도 쉽게 활용할 수 있는 기능을 제공합니다.

   한편, GOOGLE Company의 알고리즘 트레이딩은 사용자들에게 조정할 수 있는 유연

# 4. 생성 문장 필터링

In [ ]:
import numpy as np

def filter_selections_by_length(selections, std_dev_threshold=2):
    lengths = [len(sentence) for sentence in selections]
    
    # 평균과 표준편차 계산
    mean_length = np.mean(lengths)
    std_dev_length = np.std(lengths)
    
    # 허용 범위 계산
    lower_bound = mean_length - (std_dev_threshold * std_dev_length)
    upper_bound = mean_length + (std_dev_threshold * std_dev_length)
    
    # 허용 범위 내의 문장만 필터링
    filtered_selections = [sentence for sentence, length in zip(selections, lengths) 
                           if lower_bound <= length <= upper_bound]
    
    return filtered_selections

In [ ]:
filtered_selections = filter_selections_by_length(selections)

for idx, select in enumerate(filtered_selections):
    print(idx+1, '-'*100)
    print(select)

1 ----------------------------------------------------------------------------------------------------
Google Sheets에서 21일 ATR(평균 진폭 범위)을 EMA(지수 이동 평균) 스무딩 방식으로 계산할 때, Google Finance 함수를 사용하여 데이터를 추출하는 방법이 있지만, α 값의 오해로 잘못된 계산이 될 수 있습니다. EMA를 계산할 때 사용되는 α 값은 보통 2 / (N + 1)으로 설정되는데, 기간 N이 21일 경우, 간혹 α를 0.5로 잘못 설정하는 경우가 있습니다. 이렇게 하면 비정상적인 데이터 스무딩이 발생할 수 있습니다.

### 단계별 예시:

1. **데이터 입력**: A열에 날짜, B열에 종가, C열에 고가, D열에 저가 데이터를 입력합니다.

2. **TR 계산**:
   E2 셀에 다음 수식을 입력합니다 (2행부터 시작한다 가정):
   ```plaintext
   =MAX(C2-D2, ABS(C2-B1), ABS(D2-B1))
   ```
   E2 셀을 아래로 드래그하여 TR을 채웁니다.

3. **ATR 계산**: F열에서 첫 번째 ATR 값을 단순 이동 평균으로 계산합니다 (F21 셀):
   ```plaintext
   =AVERAGE(E2:E22)
   ```

4. **EMA 계산**: G열에서 잘못된 α 값 0.5를 사용하여 EMA를 계산합니다 (G22 셀에):
   ```plaintext
   =E22 * 0.5 + G21 * (1 - 0.5)
   ```
   G22 셀을 선택한 후 아래로 드래그하여 EMA 계산을 계속합니다.

5. **최종적으로**: G열의 마지막 값을 참조하여 21일 EMA 스무딩 ATR을 얻을 수 있지만, 정확도는 낮을 수 있습니다.

이 방법은 α 설정 오류로 인해 정확하지 않은 ATR 값을 산출할 가능성이 있으므로 주의가 필요합니다.
2 -------------------------------

# 3. Quality Control

In [ ]:
quality_control_prompt_system="""You are the final sensitivity reader for a benchmark that is about to be published.
Read through a question included in the benchmark and evaluate whether the question is answerable.
A question is deemed unanswerable if:

1. The question does not include the name of a company.
2. The question is based on information that is outdated or no longer relevant.
3. The question assumes knwledge that is not commonly available or requires specialized expertise beyond the scape of the benchmark audience.
4. The question's phrasing is ambiguous or can lead to multiple equally valid interpertations.
5. The information needed to answer the question correctly is not present within the context provided in the benchmark.
6. The question contains biases or assumptions that could disadvantage certain group of people or promote stereotypes.

please ensure that each question in the benchmark meets these criteria to be considered answerable and appropriate for publication. Explain wheter the question meets each criteria and return [[Yse]] for answerable question and [[No]] for unanswerable questions.
"""

quality_control_prompt_user = """
### Question: {}
### Gold Answer: {}
### Wrong Answers:
    1. {}
    2. {}
    3. {}
    4. {}

### Decision:
"""

In [ ]:
# 8.52 -> 8.42
wrong_answers = filtered_selections[:4]
qc_system_prompt = {'role':'system', 'content':quality_control_prompt_system}
qc_user_prompt = {'role':'user', 'content': quality_control_prompt_user.format(question, gold_answer, wrong_answers[0], wrong_answers[1], wrong_answers[2], wrong_answers[3])}

response = client.chat.completions.create(
        model='gpt-4o',
        messages = [qc_system_prompt, qc_user_prompt]
    )

print(response.choices[0].message.content)

### Evaluation:

1. **Company Name Inclusion**: The question specifically mentions Google (Sheets and Finance), so it includes the name of the company.  - Yes
2. **Outdated Information**: The question discusses using current functionalities of Google Sheets and its integration with Google Finance for calculating ATR via EMA smoothing, which are up-to-date.  - Yes
3. **Specialized Knowledge Beyond Scope**: Calculating ATR using EMA smoothing in Google Sheets does not typically require specialized financial expertise beyond a general understanding of spreadsheet functions and basic stock market terms like "ATR" and "EMA". However, it could be argued that understanding Pine Script and translating its functionality into a Google Sheets formula slightly bleeds into specialized knowledge. - Possibly No
4. **Ambiguous Phrasing**: The question clearly outlines what is needed, and even though it involves complex tasks, the phrasing is not ambiguous. - Yes
5. **Lack of Information for Correct An